In [ ]:
#Mount the drive in colab to be able to share outputs across the notebooks
import sys
import os
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive/')

    %mkdir -p /content/drive/MyDrive/ai-course
    %cd /content/drive/MyDrive/ai-course

    if not os.path.exists('ai-course'):
        !git clone https://github.com/jpata/ai-course
    
    %cd ai-course
    !git pull

# YOLO Fine-Tuning

This notebook demonstrates how to fine-tune a YOLO model on a custom dataset. The dataset was labeled automatically using the OWL2 model in the `module4_owl2_object_detection.md` notebook.

First, let's install the necessary libraries.

```python
#!pip install -q ultralytics pandas pyyaml scikit-learn
```

Now, let's import the required libraries.

In [ ]:
import os
#os.environ['YOLO_VERBOSE'] = 'False'
import yaml
from ultralytics import YOLO
from PIL import Image
from IPython.display import display
import pandas as pd
import numpy as np
import seaborn as sns
import tqdm

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

## Create Dataset YAML

YOLO models require a `dataset.yaml` file that specifies the dataset paths and class names. We will create this file now. The labels were generated by the OWL2 notebook and are located in `data/IDLE-OO-Camera-Traps/labels/test`. The corresponding images are in `data/IDLE-OO-Camera-Traps/data/test`.

**Note:** The `module4_owl2_object_detection.md` notebook generates labels for a sample of 500 images. Fine-tuning on a small dataset (like 500 images, which is still relatively small for object detection) will not produce a robust model, but it demonstrates the process. For better results, you should generate labels for a larger portion of the dataset.

In [ ]:
import glob
from sklearn.model_selection import train_test_split

# --- Find all labeled images and create train.txt and val.txt ---
base_path = os.path.abspath('../data/IDLE-OO-Camera-Traps_yolo')
labels_dir = os.path.join(base_path, 'labels')
images_dir = os.path.join(base_path, 'images')
train_file_path = os.path.join(base_path, 'train.txt')
val_file_path = os.path.join(base_path, 'val.txt')
image_files = []

# Recursively find all .txt files in the labels directory, excluding classes.txt
label_files = [f for f in glob.glob(os.path.join(labels_dir, '**/*.txt'), recursive=True) if os.path.basename(f) != 'classes.txt']

for label_file in label_files:
    # Derive the corresponding image path, assuming .png extension
    image_path = label_file.replace("/labels/", "/images/").replace(".txt", ".png")
    if os.path.exists(image_path):
        image_files.append(image_path)

if not image_files:
    raise Exception("No labeled images found. 'train.txt' and 'val.txt' were not created.")

# Split the data into training and validation sets (80% train, 20% val)
train_images, val_images = train_test_split(image_files, test_size=0.2, random_state=42)

# Write the relative paths of labeled images to train.txt
with open(train_file_path, 'w') as f:
    for image_path in train_images:
        f.write(f"{image_path}\n")
print(f"Found {len(image_files)} labeled images.")
print(f"Created '{train_file_path}' with {len(train_images)} images for training.")

# Write the relative paths of labeled images to val.txt
with open(val_file_path, 'w') as f:
    for image_path in val_images:
        f.write(f"{image_path}\n")
print(f"Created '{val_file_path}' with {len(val_images)} images for validation.")


# --- Create dataset.yaml ---
dataset_config = {
    'path': os.path.abspath(base_path), # Use absolute path
    'train': 'train.txt',
    'val': 'val.txt',
    'names': {}
}
classes_path = os.path.join(os.path.dirname(labels_dir), 'classes.txt')
with open(classes_path, 'r') as f:
    classes = [line.strip() for line in f.readlines()]
    dataset_config['names'] = {i: name for i, name in enumerate(classes)}

with open('ena24_yolo_dataset.yaml', 'w') as f:
    yaml.dump(dataset_config, f)

print("\nena24_yolo_dataset.yaml created:")
with open('ena24_yolo_dataset.yaml', 'r') as f:
    print(f.read())


## Fine-Tune YOLO Model

Now we can load a pretrained YOLO model and fine-tune it on our custom dataset. We'll use the `yolov8n.pt` model.

If the `ena24_yolo_dataset.yaml` was created successfully, we can proceed with training.

In [ ]:
# Load a pretrained YOLO model
model = YOLO('../yolov8n.pt')

# Train the model
results = model.train(data='ena24_yolo_dataset.yaml', epochs=100, imgsz=640, batch=8, fliplr=0.5, translate=0.1, scale=0.5)

## Visualize Training and Validation Loss

In [ ]:
# Find the latest training directory
train_dir = 'runs/detect'
latest_train_run = max(os.listdir(train_dir), key=lambda d: os.path.getmtime(os.path.join(train_dir, d)))
results_csv_path = os.path.join(train_dir, latest_train_run, 'results.csv')

print(f"Loading training results from: {results_csv_path}")
results_df = pd.read_csv(results_csv_path)

fig = plt.figure(figsize=(12, 6))
plt.plot(results_df['epoch'], results_df['train/box_loss']+results_df['train/cls_loss']+results_df['train/dfl_loss'], label='Train Loss')
plt.plot(results_df['epoch'], results_df['val/box_loss']+results_df['val/cls_loss']+results_df['val/dfl_loss'], label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Curves')
plt.legend()